In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import cv2
import keras 
from keras.utils import normalize
from keras import *
import pandas as pd
import tensorflow as tf


In [ ]:
x_train_val=np.load('x_train_val.npy')
y_train_val=np.load('y_train_val.npy')

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 13

In [ ]:
#Build the model
# The input array shape is (number of training images, IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS). 
# We used 256 x 256 image size and 13 img channels which represent the 13 predictive features
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
#s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)


In [ ]:
#Contraction path
c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(inputs)
c1 = tf.keras.layers.BatchNormalization()(c1)
c1 = tf.keras.layers.LeakyReLU(alpha=0.2)(c1)
c1 = tf.keras.layers.Dropout(0.5)(c1)
c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c1)
c1 = tf.keras.layers.BatchNormalization()(c1)
c1 = tf.keras.layers.LeakyReLU(alpha=0.2)(c1)
c1 = tf.keras.layers.Dropout(0.5)(c1)
p1 = tf.keras.layers.AveragePooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(p1)
c2 = tf.keras.layers.BatchNormalization()(c2)
c2 = tf.keras.layers.LeakyReLU(alpha=0.2)(c2)
c2 = tf.keras.layers.Dropout(0.5)(c2)
c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c2)
c2 = tf.keras.layers.BatchNormalization()(c2)
c2 = tf.keras.layers.LeakyReLU(alpha=0.2)(c2)
c2 = tf.keras.layers.Dropout(0.5)(c2)
p2 = tf.keras.layers.AveragePooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(p2)
c3 = tf.keras.layers.BatchNormalization()(c3)
c3 = tf.keras.layers.LeakyReLU(alpha=0.2)(c3)
c3 = tf.keras.layers.Dropout(0.5)(c3)
c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c3)
c3 = tf.keras.layers.BatchNormalization()(c3)
c3 = tf.keras.layers.LeakyReLU(alpha=0.2)(c3)
c3 = tf.keras.layers.Dropout(0.5)(c3)
p3 = tf.keras.layers.AveragePooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(p3)
c4 = tf.keras.layers.BatchNormalization()(c4)
c4 = tf.keras.layers.LeakyReLU(alpha=0.2)(c4)
c4 = tf.keras.layers.Dropout(0.5)(c4)
c4 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c4)
c4 = tf.keras.layers.BatchNormalization()(c4)
c4 = tf.keras.layers.LeakyReLU(alpha=0.2)(c4)
c4 = tf.keras.layers.Dropout(0.5)(c4)
p4 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(p4)
c5 = tf.keras.layers.BatchNormalization()(c5)
c5 = tf.keras.layers.LeakyReLU(alpha=0.2)(c5)
c5 = tf.keras.layers.Dropout(0.5)(c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c5)
c5 = tf.keras.layers.BatchNormalization()(c5)
c5 = tf.keras.layers.LeakyReLU(alpha=0.2)(c5)
c5 = tf.keras.layers.Dropout(0.5)(c5)


In [ ]:
#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(u6)
c6 = tf.keras.layers.BatchNormalization()(c6)
c6 = tf.keras.layers.LeakyReLU(alpha=0.2)(c6)
c6 = tf.keras.layers.Dropout(0.5)(c6)
c6 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c6)
c6 = tf.keras.layers.BatchNormalization()(c6)
c6 = tf.keras.layers.LeakyReLU(alpha=0.2)(c6)
c6 = tf.keras.layers.Dropout(0.5)(c6)

 
u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(u7)
c7 = tf.keras.layers.BatchNormalization()(c7)
c7 = tf.keras.layers.LeakyReLU(alpha=0.2)(c7)
c7 = tf.keras.layers.Dropout(0.5)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c7)
c7 = tf.keras.layers.BatchNormalization()(c7)
c7 = tf.keras.layers.LeakyReLU(alpha=0.2)(c7)
c7 = tf.keras.layers.Dropout(0.5)(c7)

u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(u8)
c8 = tf.keras.layers.BatchNormalization()(c8)
c8 = tf.keras.layers.LeakyReLU(alpha=0.2)(c8)
c8 = tf.keras.layers.Dropout(0.5)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c8)
c8 = tf.keras.layers.BatchNormalization()(c8)
c8 = tf.keras.layers.LeakyReLU(alpha=0.2)(c8)
c8 = tf.keras.layers.Dropout(0.5)(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(u9)
c9 = tf.keras.layers.BatchNormalization()(c9)
c9 = tf.keras.layers.LeakyReLU(alpha=0.2)(c9)
c9 = tf.keras.layers.Dropout(0.5)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',use_bias=False)(c9)
c9 = tf.keras.layers.BatchNormalization()(c9)
c9 = tf.keras.layers.LeakyReLU(alpha=0.2)(c9)
c9 = tf.keras.layers.Dropout(0.5)(c9)

In [ ]:
outputs = tf.keras.layers.Conv2D(1, (1, 1), padding='SAME')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])


optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')

model.compile(optimizer=optimizer, loss=['mse'], loss_weights=[1],metrics=['accuracy'])
model.summary()

In [ ]:
#Modelcheckpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard
from datetime import datetime
start_time = datetime.now()
NAME="U_Net_SA1&2"
checkpointer = ModelCheckpoint("U_Net_SA1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))



history = model.fit(x_train_val, y_train_val, validation_split=0.25, batch_size=10, epochs=500, callbacks=[checkpointer,early,tensorboard])
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
